[View in Colaboratory](https://colab.research.google.com/github/gauravsingh1012/Using-Pandas-For-Data-Analysis-and-Data-Visualisation/blob/master/Pandas_For_Finance_Chapter_1_Reshaping,_Reorganizing_and_Aggregating.ipynb)

#Chapter 1: Reshaping, Reorganizing and Aggregating



> ** A. Loading historical stock data from the quandl **



In [1]:
!pip install quandl

    100% |████████████████████████████████| 51kB 5.4MB/s 
  Running setup.py bdist_wheel for inflection ... - done
  Stored in directory: /content/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [0]:
import quandl
import pandas as pd
import numpy as np
import time

In [3]:
quandl.ApiConfig.api_key = input('Please enter your quandl Key: ')
exchange = 'WIKI'

Please enter your quandl Key: 


In [4]:
ticker = 'MSFT'
msft = quandl.get('%s/%s' % (exchange, ticker))
msft.head(1)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1986-03-13,25.5,29.25,25.5,28.0,3582600.0,0.0,1.0,0.058941,0.067609,0.058941,0.06472,1.031789e+09


In [5]:
ticker = 'AAPL'
appl = quandl.get('%s/%s' % (exchange, ticker))
appl.head(1)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.42447,0.422706,0.422706,117258400.0


In [0]:
# Set the index to a column called Date
msft = msft.reset_index(level=0)
appl = appl.reset_index(level=0)




> **B. Concatenating multiple DataFrame objects**



> To demonstrate various forms of concatenation, we will start with the following
data that shows the adjusted closing prices for MSFT for the months of January and
February 2012 represented in the following command. This dataset simulates the
retrieval of stock information representing two different time periods and stores the
data in two different DataFrame objects, as shown here:






In [0]:
msft.set_index('Date', inplace = True)
appl.set_index('Date', inplace = True)

In [8]:
msftJanClose = msft['2012-01'][['Adj. Close']]
msftFebClose = msft['2012-02'][['Adj. Close']]
msftJanClose[:3]

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435


In [9]:
msftFebClose[:3]

,Adj. Close
Date,
2012-02-01,25.453403
2012-02-02,25.504497
2012-02-03,25.751452


In [10]:
pd.concat([msftJanClose.head(3), msftFebClose.head(3)])

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435
2012-02-01,25.453403
2012-02-02,25.504497
2012-02-03,25.751452




> The resulting DataFrame contains an index identical in structure to both of the
objects, with labels from the first object and then the second object copied into the
new object. At first glance, it may appear that the concatenation is a pure copy of the
rows from each DataFrame into the new DataFrame , but as we will see, the process is
more elaborate (and hence flexible). This will become more evident as we take a look
at more examples.
The following example concatenates the first five adjusted close values in January
for both MSFT and AAPL. These have identical index labels and result in duplicate
index labels in the new DataFrame . During a concatenation along the row axis,
pandas will not align the index labels. They will be copied and this can create
duplicate, identical index labels:

In [11]:
applJanClose = appl['2012-01'][['Adj. Close']]
withDups = pd.concat([msftJanClose[:3], applJanClose[:3]])
withDups

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435
2012-01-03,52.848787
2012-01-04,53.132802
2012-01-05,53.722681


In [12]:
withDups.loc['2012-01-03']

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-03,52.848787




> This concatenation has lost whether the Adj Close value in the new DataFrame
came from the MSFT or AAPL DataFrame . This source DataFrame of each row
can be preserved during concatenation by specifying the value of the keys in the
new DataFrame . These keys will add an additional level to the index (making a
MultiIndex ), which then can be used to identify the source DataFrame :

In [13]:
closes = pd.concat([msftJanClose[:3], applJanClose[:3]],
keys=['MSFT', 'APPL'])
closes

Adj. Close
     Date                  
MSFT 2012-01-03   22.792249
     2012-01-04   23.332995
     2012-01-05   23.571435
APPL 2012-01-03   52.848787
     2012-01-04   53.132802
     2012-01-05   53.722681



> Using this new MultiIndex , it is then possible to extract the values for either stock
from this new DataFrame by only using the index labels. The following command
does this for the MSFT entries:

In [14]:
closes.loc['MSFT'][:3]

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435




> Concatenation along the row axis can also be performed using DataFrame objects
with multiple columns. The following command modifies the previous example to
use the Adj Close and Adj Volume columns in each DataFrame . 

In [0]:
msftCV = msft[['Adj. Close', 'Adj. Volume']]
applCV = appl[['Adj. Close', 'Adj. Volume']]

In [16]:
pd.concat([msftCV[:3], applCV[:3]])

,Adj. Close,Adj. Volume
Date,,
1986-03-13,0.064720,1.031789e+09
1986-03-14,0.067031,3.081600e+08
1986-03-17,0.068187,1.331712e+08
1980-12-12,0.422706,1.172584e+08
1980-12-15,0.400652,4.397120e+07
1980-12-16,0.371246,2.643200e+07




> The columns in the DataFrame objects in a concatenation do not have to have the
same names. The following command demonstrates a concatenation where the
aaplA DataFrame only consists of the Adj Close column, whereas the MSFT
DataFrame has both Adj Close and Volume columns:

In [17]:
applC = appl[['Adj. Close']]
pd.concat([msftCV[:3], applC[:3]])

,Adj. Close,Adj. Volume
Date,,
1986-03-13,0.064720,1.031789e+09
1986-03-14,0.067031,3.081600e+08
1986-03-17,0.068187,1.331712e+08
1980-12-12,0.422706,NaN
1980-12-15,0.400652,NaN
1980-12-16,0.371246,NaN




> Since the rows originating from the appl DataFrame do not have a Volume column,
pandas inserts NaN into the Volume column for those rows.
The set of columns that results from a concatenation along the row axis is the result
of relational algebra across the names of the columns. In this default scenario, the
resulting column is the union of column names from each DataFrame . This can be
changed to an intersection using the join parameter. The following command makes
the set of resulting columns the intersection of the column names by specifying
join='inner' :

In [18]:
pd.concat([msftCV[:3], applC[:3]], join='inner')

,Adj. Close
Date,
1986-03-13,0.064720
1986-03-14,0.067031
1986-03-17,0.068187
1980-12-12,0.422706
1980-12-15,0.400652
1980-12-16,0.371246




> We can change the axis for concatenation to the columns using axis=1 :

In [19]:
msftC = msft[['Adj. Close']]
closes = pd.concat([msftJanClose[:3], applJanClose[:3]], axis=1)
closes

,Adj. Close,Adj. Close
Date,,
2012-01-03,22.792249,52.848787
2012-01-04,23.332995,53.132802
2012-01-05,23.571435,53.722681




> Note that this DataFrame has two Adj Close columns and only consists of 3 rows
(the concatenation along axis=0 has 6 ). Because of the use of axis=1 , the union of
the index labels is derived instead from the column names, and the columns are copied
one by one in an orderly manner from the DataFrame objects, including duplicates.
It is also possible to concatenate with multiple columns where the DataFrame objects
do not have the same set of index labels. The following command concatenates the
first five msftCV values and the first three applCV values:

In [20]:
pd.concat([msftJanClose[:5], applJanClose[:3]], axis=1, keys=['MSFT', 'APPL'])

,MSFT,APPL
,Adj. Close,Adj. Close
Date,,
2012-01-03,22.792249,52.848787
2012-01-04,23.332995,53.132802
2012-01-05,23.571435,53.722681
2012-01-06,23.933352,NaN
2012-01-09,23.622529,NaN




> This results in duplicate column names, so we use the keys parameter to create
MultiIndex for the columns. Since there were row index labels that were not found
in aaplCV , pandas fills those with NaN .
Just as with concatenation along the row axis, the type of join performed by
pd.concat() can be changed using the join parameter. The following command
performs an inner join instead of an outer join, which results in the intersection of
row index labels:

In [21]:
pd.concat([msftJanClose[:5], applJanClose[:3]], axis=1,join='inner', keys=['MSFT', 'AAPL'])

,MSFT,AAPL
,Adj. Close,Adj. Close
Date,,
2012-01-03,22.792249,52.848787
2012-01-04,23.332995,53.132802
2012-01-05,23.571435,53.722681




> The resulting DataFrame only has three rows because those index labels were the
only ones in common in the two concatenated DataFrame objects.
If you want to ignore indices in the result of pd.concat() , you can use the ignore_
index=True parameter, which will drop the index and create a default zero-based
integer index, as shown here:

In [22]:
pd.concat([msftJanClose[:5], applJanClose[:3]], ignore_index=True)

,Adj. Close
0,22.792249
1,23.332995
2,23.571435
3,23.933352
4,23.622529
5,52.848787
6,53.132802
7,53.722681




> **C. Merging DataFrame objects**


> The combination of pandas objects is allowed using relational database-like join
operations, high-performance in-memory operations, and the pd.merge() function.
Merging in pandas differs from concatenation in that the pd.merge() function
combines data based on the values of the data in one or more columns instead of
using the index label values along a specific axis.
The default process that pd.merge() uses is to first identify the columns the data of
which will be used in the merge, and then to perform an inner join based upon that
information. The columns used in the join are, by default, selected as those in both
DataFrame objects with common names (an intersection of the column labels).
To demonstrate a merge, we will use the following two DataFrame objects, one with
the volumes and the other with the adjusted close values for MSFT. Both have the
index reset:



In [0]:
msftVR = msft[['Adj. Volume']].reset_index()
msftCR = msft[['Adj. Close']].reset_index()


In [24]:
msftVR[:3]

,Date,Adj. Volume
0,1986-03-13,1.031789e+09
1,1986-03-14,3.081600e+08
2,1986-03-17,1.331712e+08


In [25]:
msftCR[:3]

,Date,Adj. Close
0,1986-03-13,0.064720
1,1986-03-14,0.067031
2,1986-03-17,0.068187


In [26]:
msftCVR = pd.merge(msftCR, msftVR)
msftCVR[:5]

,Date,Adj. Close,Adj. Volume
0,1986-03-13,0.064720,1.031789e+09
1,1986-03-14,0.067031,3.081600e+08
2,1986-03-17,0.068187,1.331712e+08
3,1986-03-18,0.066454,6.776640e+07
4,1986-03-19,0.065298,4.789440e+07




> Since both msftCR, msftVR have their indexes reset to level=0. Instead of using Date as the index, these have Date as a column so that it can be used
in the merge. Our goal is to create a DataFrame that contains a Date column and
both AdjClose and AdjVolume columns. This has been accomplished with the above statement. 


> The column in common is Date ; therefore, pandas performs an inner join on the
values in that column across both DataFrame objects. Once that set is calculated,
pandas copies in the appropriate values for each row from both DataFrame objects.
The types of joins supported by pd.merge() are similar to the different types of joins
supported in relational databases. They are as follows:



> 1.   left : Use keys from the left DataFrame (equivalent to SQL's left-outer join)
2.   right : Use keys from the right DataFrame (equivalent to SQL's right-outer
join)
3.   outer : Use the union of keys from both DataFrame objects (equivalent to
SQL's full outer join)
4.    inner : Use the intersection of keys from both DataFrame objects (equivalent
to SQL's inner join)



> To demonstrate each difference in the results between inner and outer joins, we will
use the following data:




In [0]:
msftCR0_5 = msftCR[0:5]
msftVR2_4 = msftVR[2:4]

In [28]:
msftCR0_5

,Date,Adj. Close
0,1986-03-13,0.064720
1,1986-03-14,0.067031
2,1986-03-17,0.068187
3,1986-03-18,0.066454
4,1986-03-19,0.065298


In [29]:
msftVR2_4

,Date,Adj. Volume
2,1986-03-17,133171200.0
3,1986-03-18,67766400.0


In [30]:
pd.merge(msftCR0_5, msftVR2_4)

,Date,Adj. Close,Adj. Volume
0,1986-03-17,0.068187,133171200.0
1,1986-03-18,0.066454,67766400.0




> as shown above, For an inner join, since there are only two rows with matching dates, the result only
has two rows and merges the DataFrame objects where Date values are in common.


>   This can be changed to an outer join with how='outer' . All rows from the outer
DataFrame are returned ( msftCR0_5 ), and values not found in the inner DataFrame
( msftVR2_4 ) are replaced with NaN :



In [31]:
pd.merge(msftCR0_5, msftVR2_4, how='outer')

,Date,Adj. Close,Adj. Volume
0,1986-03-13,0.064720,NaN
1,1986-03-14,0.067031,NaN
2,1986-03-17,0.068187,133171200.0
3,1986-03-18,0.066454,67766400.0
4,1986-03-19,0.065298,NaN




> **D. Pivoting**


> Financial data is often stored in a format where the data is not normalized and,
therefore, has repeated values in many columns or values that logically should exist
in other tables. An example of this would be the following, where the historical
prices for multiple stocks are represented in a single DataFrame using a Symbol
column. The following command creates a DataFrame with this schema and
populates the records:




In [32]:
msftJan = msft['2012-01']
applJan = appl['2012-01']
msftJan.insert(0, 'Symbol', 'MSFT')
applJan.insert(0, 'Symbol', 'APPL')
combined = pd.concat([msftJan, applJan]).sort_index()
janData = combined.reset_index();
janData[:6]

,Date,Symbol,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2012-01-03,MSFT,26.5500,26.960,26.39,26.765,64731500.0,0.0,1.0,22.609162,22.958305,22.472910,22.792249,64731500.0
1,2012-01-03,APPL,409.4000,412.500,409.00,411.230,10793600.0,0.0,1.0,52.613606,53.011999,52.562200,52.848787,75555200.0
2,2012-01-04,MSFT,26.8199,27.470,26.78,27.400,80516100.0,0.0,1.0,22.839000,23.392605,22.805022,23.332995,80516100.0
3,2012-01-04,APPL,410.0000,414.680,409.28,413.440,9286500.0,0.0,1.0,52.690714,53.292160,52.598184,53.132802,65005500.0
4,2012-01-05,MSFT,27.3800,27.728,27.29,27.680,56081400.0,0.0,1.0,23.315964,23.612310,23.239323,23.571435,56081400.0
5,2012-01-05,APPL,414.9500,418.550,412.67,418.030,9688200.0,0.0,1.0,53.326858,53.789509,53.033847,53.722681,67817400.0




> Now let's suppose we want to extract, from this DataFrame , a new DataFrame that
is indexed by date and has columns representing the Adj. Close value for all of the
stocks listed in the Symbol column. This can be performed using the .pivot()
method of the DataFrame :



In [0]:
closes = janData.pivot(index='Date', columns='Symbol', values='Adj. Close')

In [34]:
closes[:3]

Symbol,APPL,MSFT
Date,,
2012-01-03,52.848787,22.792249
2012-01-04,53.132802,23.332995
2012-01-05,53.722681,23.571435




> This has taken all the distinct values from the Symbol column, pivoted them into
columns on the new DataFrame , and then entered the values in those columns from
the Adj. Close value for the specific symbol from the original DataFrame .




> **E. Stacking and unstacking**


> The DataFrame methods similar in operation to the pivot function are .stack() and
.unstack() . Stacking unpivots the column labels into another level of the index. To
demonstrate this, the following command pivots the MSFT and APPL columns into
the index:





In [35]:
stackedCloses = closes.stack()
stackedCloses

Date        Symbol
2012-01-03  APPL      52.848787
            MSFT      22.792249
2012-01-04  APPL      53.132802
            MSFT      23.332995
2012-01-05  APPL      53.722681
            MSFT      23.571435
2012-01-06  APPL      54.284287
            MSFT      23.933352
2012-01-09  APPL      54.198183
            MSFT      23.622529
2012-01-10  APPL      54.392239
            MSFT      23.707686
2012-01-11  APPL      54.303564
            MSFT      23.605497
2012-01-12  APPL      54.154488
            MSFT      23.843937
2012-01-13  APPL      53.951436
            MSFT      24.056829
2012-01-17  APPL      54.579869
            MSFT      24.061087
2012-01-18  APPL      55.146616
            MSFT      24.039798
2012-01-19  APPL      54.971837
            MSFT      23.946125
2012-01-20  APPL      54.014408
            MSFT      25.300120
2012-01-23  APPL      54.928014
            MSFT      25.317152
2012-01-24  APPL      54.028545
            MSFT      24.985040
2012-01-25  APPL     



> This has created a new index with an additional level named Symbol (from the name
of the columns index). Each row is then indexed by both Date and Symbol . And for
each unique Date and Symbol level, pandas has inserted the appropriate Adj Close
value.



> The result of this allows the efficient lookup of any Adj. Close value using the index.
To look up the Adj Close value for APPL on 2012-01-03 , we can use the following
command:





In [36]:
stackedCloses.loc[('2012-01-03','APPL')] # Syntactically integrating MultiIndex 
#in advanced indexing with .loc is a bit challenging, but we’ve made every effort to do so. 
#In general, MultiIndex keys take the form of tuples. 

52.848786580038



> The result here is equivalent to the following value-based lookup, but is significantly
more efficient, uses less typing, and is also organized better, causing less mental
clutter.
Using a MultiIndex , it is also possible to look up values for just a specific Date :



In [37]:
stackedCloses.loc['2012-01-03']

Date        Symbol
2012-01-03  APPL      52.848787
            MSFT      22.792249
dtype: float64



> For a specific Symbol , here is the command:



In [38]:
stackedCloses.loc[:, 'MSFT']

Date
2012-01-03    22.792249
2012-01-04    23.332995
2012-01-05    23.571435
2012-01-06    23.933352
2012-01-09    23.622529
2012-01-10    23.707686
2012-01-11    23.605497
2012-01-12    23.843937
2012-01-13    24.056829
2012-01-17    24.061087
2012-01-18    24.039798
2012-01-19    23.946125
2012-01-20    25.300120
2012-01-23    25.317152
2012-01-24    24.985040
2012-01-25    25.172385
2012-01-26    25.121291
2012-01-27    24.891367
2012-01-30    25.214963
2012-01-31    25.146838
dtype: float64



> The .unstack() method performs the opposite function; that is, it pivots a level of
an index into a column in a new DataFrame . The following command unstacks the
last level of the MultiIndex and results in a DataFrame equivalent to the original
unstackedCloses :



In [39]:
unstackedCloses = stackedCloses.unstack()
unstackedCloses[:3]

Symbol,APPL,MSFT
Date,,
2012-01-03,52.848787,22.792249
2012-01-04,53.132802,23.332995
2012-01-05,53.722681,23.571435




> **F. Melting**


> Melting is the process of transforming a DataFrame into a format where each
row represents a unique id-variable combination. The following command
demonstrates melting the janData DataFrame into an id-variable combination
consisting of the Date and Symbol columns as the ID and the other columns mapped
into the variables:

In [40]:
melted = pd.melt(janData, id_vars=['Date', 'Symbol'])
melted[:5]

,Date,Symbol,variable,value
0,2012-01-03,MSFT,Open,26.5500
1,2012-01-03,APPL,Open,409.4000
2,2012-01-04,MSFT,Open,26.8199
3,2012-01-04,APPL,Open,410.0000
4,2012-01-05,MSFT,Open,27.3800




> During a melt, the column(s) specified by the id_vars parameter remain columns
(in this case Date and Symbol ). All other columns have their names mapped to the
values in the variable column—one row for each variable column of an id_var
column value combination.
This organization of data is useful to select chunks of information based upon a
specific ID variable and then one or more variables. As an example, the following
command returns all measurements for 2012-01-03 and the MSFT symbol:



In [41]:
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')]

,Date,Symbol,variable,value
0,2012-01-03,MSFT,Open,2.655000e+01
40,2012-01-03,MSFT,High,2.696000e+01
80,2012-01-03,MSFT,Low,2.639000e+01
120,2012-01-03,MSFT,Close,2.676500e+01
160,2012-01-03,MSFT,Volume,6.473150e+07
200,2012-01-03,MSFT,Ex-Dividend,0.000000e+00
240,2012-01-03,MSFT,Split Ratio,1.000000e+00
280,2012-01-03,MSFT,Adj. Open,2.260916e+01
320,2012-01-03,MSFT,Adj. High,2.295830e+01
360,2012-01-03,MSFT,Adj. Low,2.247291e+01


In [42]:
#while same operation on original dataframe of JanData would have revealed a subset dataframe based on boolean op:
janData[(janData.Date=='2012-01-03') & (janData.Symbol=='MSFT')]

,Date,Symbol,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2012-01-03,MSFT,26.55,26.96,26.39,26.765,64731500.0,0.0,1.0,22.609162,22.958305,22.47291,22.792249,64731500.0




> **G. Grouping and aggregating**


> Data in pandas can be easily split into groups and then summarized using various
statistical and quantitative calculations. This process in pandas nomenclature is often
referred to as the **split-apply-combine pattern**.
In this section, we will look at using this pattern as applied to stock data. We will
split the data by various time and symbol combinations and then apply statistical
operations to begin analyzing the risk and return on our sample data.


> 1. **Splitting**: 
Objects in pandas are split into groups using the . groupby() method. To
demonstrate this, we will use the stock price data introduced earlier in the chapter
but slightly reorganized to facilitate understanding of the grouping process:





In [0]:
JanSC = combined[['Symbol', 'Adj. Close']].reset_index()
JanSC.insert(1, 'Year', pd.DatetimeIndex(JanSC['Date']).year)
JanSC.insert(2, 'Month',pd.DatetimeIndex(JanSC['Date']).month)

In [44]:
JanSC[:6]

,Date,Year,Month,Symbol,Adj. Close
0,2012-01-03,2012,1,MSFT,22.792249
1,2012-01-03,2012,1,APPL,52.848787
2,2012-01-04,2012,1,MSFT,23.332995
3,2012-01-04,2012,1,APPL,53.132802
4,2012-01-05,2012,1,MSFT,23.571435
5,2012-01-05,2012,1,APPL,53.722681




> This data differs from before as only the AdjClose value is utilized, and the Date
column is broken apart into two other columns, Year and Month . This splitting of the
date is done to be able to provide the ability to group the data by Month and Year for
each Symbol variable.


> This data consists of four categorical variables ( Date , Symbol , Year , and Month ) and
one continuous variable, AdjClose . In pandas, it is possible to group by any single
categorical variable by passing its name to .groupby() . The following command
groups by the Symbol column:



In [45]:
JanSC.groupby('Symbol')



> The result of calling .groupby() on a DataFrame is not the actual grouped data but
a DataFrameGroupBy object (a SeriesGroupBy for a grouping on a Series ). The
grouping has not actually been performed yet as grouping is a deferred/lazy process
in pandas.


> This result of .groupby() is a subclass of a GroupBy object and is an interim
description of the grouping to be performed (if you are a C# programmer, this feels
a lot like an expression tree created by LINQ). This allows pandas to first validate
that the grouping description provided to it is valid relative to the data before the
processing starts.


> There are number of useful properties on a GroupBy object. The .groups property
will return a Python dictionary whose keys represent the name of each group (if
multiple columns are specified, it is a tuple), and the values are an array of the index
labels contained within each group:



In [46]:
grouped = JanSC.groupby('Symbol')
type(grouped.groups)

dict

In [47]:
grouped.groups

{'APPL': Int64Index([1, 3, 5, 7, 9, 10, 12, 14, 16, 19, 20, 23, 24, 26, 29, 30, 32, 34,
             36, 39],
            dtype='int64'),
 'MSFT': Int64Index([0, 2, 4, 6, 8, 11, 13, 15, 17, 18, 21, 22, 25, 27, 28, 31, 33, 35,
             37, 38],
            dtype='int64')}

In [48]:
len(grouped), grouped.ngroups

(2, 2)



> Splitting is not performed until you take some type of action on the GroupBy object.
It is, however, possible to iterate over several properties of the object to view how the
data will be grouped (hence forcing it to be grouped). The following helper function
demonstrates this and will be used frequently throughout this chapter:



In [0]:
def print_groups (groupobject):
  for name, group in groupobject:
    print(name)
    print(group.head())

In [50]:
print_groups(grouped)

APPL
        Date  Year  Month Symbol  Adj. Close
1 2012-01-03  2012      1   APPL   52.848787
3 2012-01-04  2012      1   APPL   53.132802
5 2012-01-05  2012      1   APPL   53.722681
7 2012-01-06  2012      1   APPL   54.284287
9 2012-01-09  2012      1   APPL   54.198183
MSFT
        Date  Year  Month Symbol  Adj. Close
0 2012-01-03  2012      1   MSFT   22.792249
2 2012-01-04  2012      1   MSFT   23.332995
4 2012-01-05  2012      1   MSFT   23.571435
6 2012-01-06  2012      1   MSFT   23.933352
8 2012-01-09  2012      1   MSFT   23.622529




> Looking at these results gives us some insight into what pandas is doing with this
specific splitting operation. It has created, for each distinct value in the Symbol
column of the original DataFrame , a group consisting of a DataFrame (this is
different from the functionality provided by itertools.groupby , so be careful
if you are used to using that library for this functionality). It then copies the non-
grouped columns and data into each of those DataFrame objects and then uses the
values from the specified column(s) as the group name.


> The .size() method of the object gives a nice summary of the size of all the groups:




In [51]:
grouped.size()

Symbol
APPL    20
MSFT    20
dtype: int64



> If you want the data for the items in any given group, you can use the .get_group()
property. The following command retrieves the MSFT group:



In [52]:
grouped.get_group('MSFT')

,Date,Year,Month,Symbol,Adj. Close
0,2012-01-03,2012,1,MSFT,22.792249
2,2012-01-04,2012,1,MSFT,23.332995
4,2012-01-05,2012,1,MSFT,23.571435
6,2012-01-06,2012,1,MSFT,23.933352
8,2012-01-09,2012,1,MSFT,23.622529
11,2012-01-10,2012,1,MSFT,23.707686
13,2012-01-11,2012,1,MSFT,23.605497
15,2012-01-12,2012,1,MSFT,23.843937
17,2012-01-13,2012,1,MSFT,24.056829
18,2012-01-17,2012,1,MSFT,24.061087




> Grouping can be performed upon multiple columns by passing a list of column
names. The following command groups the data by the Symbol and Year and Month
variables:



In [53]:
mcg = JanSC.groupby(['Symbol', 'Year', 'Month'])
print_groups(mcg)

('APPL', 2012, 1)
        Date  Year  Month Symbol  Adj. Close
1 2012-01-03  2012      1   APPL   52.848787
3 2012-01-04  2012      1   APPL   53.132802
5 2012-01-05  2012      1   APPL   53.722681
7 2012-01-06  2012      1   APPL   54.284287
9 2012-01-09  2012      1   APPL   54.198183
('MSFT', 2012, 1)
        Date  Year  Month Symbol  Adj. Close
0 2012-01-03  2012      1   MSFT   22.792249
2 2012-01-04  2012      1   MSFT   23.332995
4 2012-01-05  2012      1   MSFT   23.571435
6 2012-01-06  2012      1   MSFT   23.933352
8 2012-01-09  2012      1   MSFT   23.622529




> Since multiple columns were specified, the name of each group is now a tuple with
the value from Symbol , Year , and Month that represents the group.
The examples up to this point have used a DataFrame without any specific indexing
(just the default sequential numerical index).

> This type of data would actually be 
very well suited for a hierarchical index, which can then be used directly to group
the data based upon index label(s). To demonstrate this, the following command
creates a new DataFrame with a MultiIndex consisting of the original Symbol , Year ,
and Month columns:



In [54]:
mi = JanSC.set_index(['Symbol', 'Year', 'Month'])
mi

,,,Date,Adj. Close
Symbol,Year,Month,,
MSFT,2012,1,2012-01-03,22.792249
APPL,2012,1,2012-01-03,52.848787
MSFT,2012,1,2012-01-04,23.332995
APPL,2012,1,2012-01-04,53.132802
MSFT,2012,1,2012-01-05,23.571435
APPL,2012,1,2012-01-05,53.722681
MSFT,2012,1,2012-01-06,23.933352
APPL,2012,1,2012-01-06,54.284287
MSFT,2012,1,2012-01-09,23.622529




> Grouping can now be performed using the levels of the hierarchical index. The
following groups by the index level 0 ( Symbol names):



In [55]:
mig_l1 = mi.groupby(level=0)
print_groups(mig_l1)

APPL
                        Date  Adj. Close
Symbol Year Month                       
APPL   2012 1     2012-01-03   52.848787
            1     2012-01-04   53.132802
            1     2012-01-05   53.722681
            1     2012-01-06   54.284287
            1     2012-01-09   54.198183
MSFT
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2012 1     2012-01-03   22.792249
            1     2012-01-04   23.332995
            1     2012-01-05   23.571435
            1     2012-01-06   23.933352
            1     2012-01-09   23.622529




> Grouping by multiple levels can be performed by passing multiple levels to
.groupby() . Also, if the MultiIndex has names specified, then those names can be
used instead of the integers for the levels. The following command groups the three
levels of the MultiIndex by using their names:



In [56]:
mig_l12 = mi.groupby(level=['Symbol', 'Year', 'Month'])
print_groups(mig_l12)

('APPL', 2012, 1)
                        Date  Adj. Close
Symbol Year Month                       
APPL   2012 1     2012-01-03   52.848787
            1     2012-01-04   53.132802
            1     2012-01-05   53.722681
            1     2012-01-06   54.284287
            1     2012-01-09   54.198183
('MSFT', 2012, 1)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2012 1     2012-01-03   22.792249
            1     2012-01-04   23.332995
            1     2012-01-05   23.571435
            1     2012-01-06   23.933352
            1     2012-01-09   23.622529




> 2.   Aggregating
Armed with the capability to group the stock data on a monthly basis, we can now
start to drive analysis of the data. Specifically, we will develop methods to calculate
the risk on a stock based on a time-window of a calendar month.
Aggregation is performed using the .aggregate() , or in short .agg() , method of
the GroupBy object. The parameter set to .agg() is a reference to a function that is
applied to each group. The following command will calculate the mean of the values
across each Symbol , Year , and Month :





In [57]:
mig_l12.agg(np.mean)

,,,Adj. Close
Symbol,Year,Month,
APPL,2012,1,55.078240
MSFT,2012,1,24.283134




> > The result of the aggregation will have an identically structured index as the original
data. If you do not want this to happen, you can use the as_index=False option of
the .groupby() method to specify not to duplicate the structure of the index, which
may be convenient in several situations, including where a function expects the data
with a numerical index:





In [59]:
JanSC.groupby(['Symbol', 'Year', 'Month'],as_index=False).agg(np.mean)[:5]

,Symbol,Year,Month,Adj. Close
0,APPL,2012,1,55.078240
1,MSFT,2012,1,24.283134


>>This has derived the same results, but there is a slightly different organization.
It is possible to apply multiple aggregation functions to each group in a single call to
.agg() by passing them in a list:

In [60]:
mig_l12.agg([np.mean, np.std])

Adj. Close          
                        mean       std
Symbol Year Month                     
APPL   2012 1      55.078240  1.714656
MSFT   2012 1      24.283134  0.778694